In [8]:
import pandas as pd
from IPython.display import display

df = pd.read_csv("merged_df_clean.csv", low_memory=False)
df

,State,StateAbbrev,StateFips,SchYear,DataLevel,DistName,SchName,NCESDistrictID,StateAssignedDistID,NCESSchoolID,...,Total Dropouts Grades 9-12,Total Revenue (TOTALREV) per Pupil (V33),Total Students All Grades (Excludes AE),Two or More Races - female,Two or More Races - male,White - female,White - male,Per Pupil Expenditure,Year_clean,Grade_numeric
0,Indiana,IN,18,2022-23,District,21St Century Charter Sch of Gary,All Schools,1800046.0,9545.0,NaN,...,‡,NaN,1337,18,11,1,1,NaN,2023.0,3.0
1,Indiana,IN,18,2022-23,District,21St Century Charter Sch of Gary,All Schools,1800046.0,9545.0,NaN,...,‡,NaN,1337,18,11,1,1,NaN,2023.0,4.0
2,Indiana,IN,18,2022-23,District,21St Century Charter Sch of Gary,All Schools,1800046.0,9545.0,NaN,...,‡,NaN,1337,18,11,1,1,NaN,2023.0,5.0
3,Indiana,IN,18,2022-23,District,21St Century Charter Sch of Gary,All Schools,1800046.0,9545.0,NaN,...,‡,NaN,1337,18,11,1,1,NaN,2023.0,6.0
4,Indiana,IN,18,2022-23,District,21St Century Charter Sch of Gary,All Schools,1800046.0,9545.0,NaN,...,‡,NaN,1337,18,11,1,1,NaN,2023.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99525,Hawaii,HI,15,2014-15,District,Hawaii Department of Education,All Schools,1500030.0,HI-001,NaN,...,NaN,14824,182384,9604,10593,11493,12811,NaN,2015.0,4.0
99526,Hawaii,HI,15,2014-15,District,Hawaii Department of Education,All Schools,1500030.0,HI-001,NaN,...,NaN,14824,182384,9604,10593,11493,12811,NaN,2015.0,5.0
99527,Hawaii,HI,15,2014-15,District,Hawaii Department of Education,All Schools,1500030.0,HI-001,NaN,...,NaN,14824,182384,9604,10593,11493,12811,NaN,2015.0,6.0
99528,Hawaii,HI,15,2014-15,District,Hawaii Department of Education,All Schools,1500030.0,HI-001,NaN,...,NaN,14824,182384,9604,10593,11493,12811,NaN,2015.0,7.0


In [ ]:
# Identifying columns with missing values
missing_counts = df.isnull().sum()
missing_columns = missing_counts[missing_counts > 0].reset_index()
missing_columns.columns = ["column", "missing_count"]

num_missing_cols = missing_columns.shape[0]

print(f"Number of columns with missing values: {num_missing_cols}")
display(missing_columns)

Number of columns with missing values: 49


,column,missing_count
0,NCESSchoolID,94369
1,StateAssignedSchID,99530
2,StudentGroup_TotalTested,18343
3,StudentSubGroup_TotalTested,18343
4,Lev1_count,24157
5,Lev1_percent,16702
6,Lev2_count,43074
7,Lev2_percent,35618
8,Lev3_count,45314
9,Lev3_percent,39172


In [14]:
# Listing numeric columns to impute
numeric_cols = [
    "StudentGroup_TotalTested",
    "ProficientOrAbove_percent",
    "Pupil/Teacher Ratio",
    "Total Revenue (TOTALREV) per Pupil (V33)",
    "Per Pupil Expenditure",
    "ParticipationRate",
]

# Converting to numeric, coercing errors
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Imputing within each district
df[numeric_cols] = df.groupby("DistrictName_clean")[numeric_cols].transform(
    lambda grp: grp.fillna(grp.mean())
)

# For entries still missing, imputing within each state
df[numeric_cols] = df.groupby("State")[numeric_cols].transform(
    lambda grp: grp.fillna(grp.mean())
)

# Finally, filling any remaining missing with overall mean
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Rounding according to requirements
df["Pupil/Teacher Ratio"] = df["Pupil/Teacher Ratio"].round(2)
df["ParticipationRate"] = df["ParticipationRate"].round(3)
df["Total Revenue (TOTALREV) per Pupil (V33)"] = df[
    "Total Revenue (TOTALREV) per Pupil (V33)"
].round(1)
df["Per Pupil Expenditure"] = df["Per Pupil Expenditure"].round(1)
df["ProficientOrAbove_percent"] = df["ProficientOrAbove_percent"].round(3)

# Converting to percentage
df['ProficientOrAbove_percent'] = df['ProficientOrAbove_percent'] * 100
df['ParticipationRate'] = df['ParticipationRate'] * 100 

# Confirming no missing values remain
missing_post = df[numeric_cols].isnull().sum()
print("Missing values after hierarchical imputation:\n", missing_post)

print(
    df[
        [
            "Pupil/Teacher Ratio",
            "ParticipationRate",
            "Total Revenue (TOTALREV) per Pupil (V33)",
            "Per Pupil Expenditure",
            "ProficientOrAbove_percent",
        ]
    ].head()
)

Missing values after hierarchical imputation:
 StudentGroup_TotalTested                    0
ProficientOrAbove_percent                   0
Pupil/Teacher Ratio                         0
Total Revenue (TOTALREV) per Pupil (V33)    0
Per Pupil Expenditure                       0
ParticipationRate                           0
dtype: int64
   Pupil/Teacher Ratio  ParticipationRate  \
0                11.63            31100.0   
1                11.63            31100.0   
2                11.63            31100.0   
3                11.63            31100.0   
4                11.63            31100.0   

   Total Revenue (TOTALREV) per Pupil (V33)  Per Pupil Expenditure  \
0                                   12330.0                12313.7   
1                                   12330.0                12313.7   
2                                   12330.0                12313.7   
3                                   12330.0                12313.7   
4                                   12330.0

In [15]:
df.shape

(99530, 78)

In [17]:
display(df.head())

,State,StateAbbrev,StateFips,SchYear,DataLevel,DistName,SchName,NCESDistrictID,StateAssignedDistID,NCESSchoolID,...,Total Dropouts Grades 9-12,Total Revenue (TOTALREV) per Pupil (V33),Total Students All Grades (Excludes AE),Two or More Races - female,Two or More Races - male,White - female,White - male,Per Pupil Expenditure,Year_clean,Grade_numeric
0,Indiana,IN,18,2022-23,District,21St Century Charter Sch of Gary,All Schools,1800046.0,9545.0,NaN,...,‡,12330.0,1337,18,11,1,1,12313.7,2023.0,3.0
1,Indiana,IN,18,2022-23,District,21St Century Charter Sch of Gary,All Schools,1800046.0,9545.0,NaN,...,‡,12330.0,1337,18,11,1,1,12313.7,2023.0,4.0
2,Indiana,IN,18,2022-23,District,21St Century Charter Sch of Gary,All Schools,1800046.0,9545.0,NaN,...,‡,12330.0,1337,18,11,1,1,12313.7,2023.0,5.0
3,Indiana,IN,18,2022-23,District,21St Century Charter Sch of Gary,All Schools,1800046.0,9545.0,NaN,...,‡,12330.0,1337,18,11,1,1,12313.7,2023.0,6.0
4,Indiana,IN,18,2022-23,District,21St Century Charter Sch of Gary,All Schools,1800046.0,9545.0,NaN,...,‡,12330.0,1337,18,11,1,1,12313.7,2023.0,7.0


In [13]:
df.to_csv("preprocessed_dataset.csv", index=False)